In [1]:
!pip install "dask[complete]"
!pip install pyarrow pandas
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.9/205.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path
import os
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
import dask.dataframe as dd
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [4]:
data_dir = Path('/content/drive/MyDrive/Data_hasking_without_banker_account')
ddf = dd.read_parquet(str(data_dir / '*.parquet'))
df = ddf.compute()

In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
df.head()

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
709,af3e6c0fbc6207736f6c97671d63e75cfc109129d673881416e12fa47f8cf5d8,CASA,3fdb8894344ab1289cec0efdd03ef870cfc23459f2e961fc1b3158d54f078b94,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011219361,VND,414.5,414.5,110000,...,XX.XX.XXXX sXXXXXXXX bcXXXX,N,21|45,c016c5efb5b3fe72156e1fed3593384def261de1811b3f5af1225633a81eeeb1,STP,HOI SO SHB,LP,CITAD SHB,NaN,20230801
894,c8ea05c1c35e2ed34d5b1d3c32950cc8fb99074b92d251a1a8788dab9750c40a,CASA,526978799ca402ab0f99c4c565917976bd22ee37dfb6f20885cf12773b260bc9,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011174207,VND,-510.0,-510.0,110000,...,<NA>,N,1|1,b19d1ca799a45a0483e0a49d0eb7504ad6d8012f8bf23374c34b166dc5ead6e0,MOB,HOI SO SHB,A2,TRAN CHI AN,NaN,20230801
902,2bdb75e6139a7717939d04a1ef7b8a7e17fe3261064a309c02aac70c7a447709,CASA,c60feae21e6d3a79d1b9dca82edf74b97b9e0bd93c461c51b092b7bdbfe8777d,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011205699,VND,500.0,500.0,110000,...,<NA>,N,1|2,8ea1260216b3cb219d1f797e1b678a71563151d09695fc3ae74db637b9933a02,MOB,HOI SO SHB,A2,VU THU HIEN,NaN,20230801
904,c8ea05c1c35e2ed34d5b1d3c32950cc8fb99074b92d251a1a8788dab9750c40a,CASA,526978799ca402ab0f99c4c565917976bd22ee37dfb6f20885cf12773b260bc9,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011192048,VND,1000.0,1000.0,110000,...,<NA>,N,1|2,adb8fa7e91207bdb4066688c0691a483fc7a60cb3e065d5f3f9361500a1bc727,MOB,HOI SO SHB,A2,NGUYEN TIEN SY,NaN,20230801
907,2bdb75e6139a7717939d04a1ef7b8a7e17fe3261064a309c02aac70c7a447709,CASA,c60feae21e6d3a79d1b9dca82edf74b97b9e0bd93c461c51b092b7bdbfe8777d,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011167253,VND,-500.0,-500.0,110000,...,<NA>,N,1|1,a390b60d34d64407a224af176bb5f2fd44994e48b7a6e469edbf468b2dab4e63,MOB,HOI SO SHB,A2,VU THU HIEN,NaN,20230801


In [7]:
df.dtypes

CUST_CUSTNO              string[pyarrow]
ACC_BUSINESSTYPE         string[pyarrow]
ACC_ACCNO                string[pyarrow]
ACC_BUSINESSNO           string[pyarrow]
ACC_CURRENCYISO          string[pyarrow]
BUSINESSNO_TRANS         string[pyarrow]
FK_CURRENCY              string[pyarrow]
AMOUNT                           float64
AMOUNTORIG                       float64
BRANCH_OFFICE            string[pyarrow]
CONTRA_ACCNO             string[pyarrow]
CONTRA_ZIP               string[pyarrow]
CONTRA_NAME              string[pyarrow]
CSHYN                    string[pyarrow]
REASON1                  string[pyarrow]
REASON2                  string[pyarrow]
REASON4                  string[pyarrow]
TR_FLAG_01               string[pyarrow]
ANALYTICAL_TRANS_CODE    string[pyarrow]
TR_SP_01                 string[pyarrow]
TR_SP_02                 string[pyarrow]
TR_SP_03                 string[pyarrow]
TR_SP_04                 string[pyarrow]
TR_SP_05                 string[pyarrow]
TR_SP_10        

In [8]:
# Optimize missing value handling
df = df.dropna(thresh=len(df) * 0.5, axis=1)  # Drop columns with more than 50% missing values
df = df.fillna(df.mode().iloc[0])  # Fill missing values with mode for string columns

# Identify numeric columns
numeric_cols = df.select_dtypes(include='float').columns

# Fill missing values with median for numeric columns only
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

In [9]:
df.isnull().sum()

CUST_CUSTNO              0
ACC_BUSINESSTYPE         0
ACC_ACCNO                0
ACC_BUSINESSNO           0
ACC_CURRENCYISO          0
BUSINESSNO_TRANS         0
FK_CURRENCY              0
AMOUNT                   0
AMOUNTORIG               0
BRANCH_OFFICE            0
CONTRA_ACCNO             0
CONTRA_ZIP               0
CONTRA_NAME              0
CSHYN                    0
REASON1                  0
REASON2                  0
TR_FLAG_01               0
ANALYTICAL_TRANS_CODE    0
TR_SP_01                 0
TR_SP_02                 0
TR_SP_03                 0
TR_SP_04                 0
TR_SP_05                 0
VALUEDATE                0
dtype: int64

In [10]:
# Label encode string columns
label_encoders = {}
for column in df.select_dtypes(include=['string']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [11]:
df

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON1,REASON2,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,VALUEDATE
709,307853,0,124557,0,12,57951,12,414.5,414.5,0,...,712708,801689,0,5,4274211,11,63,2,15985,0
894,353656,0,160657,0,12,12797,12,-510.0,-510.0,0,...,424297,674913,0,1,3952059,7,63,0,188373,0
902,76980,0,385399,0,12,44289,12,500.0,500.0,0,...,594131,229221,0,2,3174201,7,63,0,220605,0
904,353656,0,160657,0,12,30638,12,1000.0,1000.0,0,...,385402,518268,0,2,3865242,7,63,0,153984,0
907,76980,0,385399,0,12,5843,12,-500.0,-500.0,0,...,163310,621458,0,1,3639232,7,63,0,220605,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031051,423673,0,10543,0,12,11269503,12,-5700.0,-5700.0,0,...,356078,518514,0,1,814652,7,63,0,168775,179
1031052,62869,0,10543,0,12,11321626,12,2500.0,2500.0,0,...,310915,180154,0,2,2008719,7,63,0,119744,179
1031056,395105,0,10543,0,12,11270712,12,-1000.0,-1000.0,0,...,553774,782221,0,1,4519756,7,63,0,206472,179
1031060,400980,0,10543,0,12,11324157,12,13387.1,13387.1,0,...,78003,452681,1,2,2885479,8,63,0,178409,179


In [12]:
# Apply FP-Growth algorithm with a support threshold of 0.10 (10%)
frequent_itemsets= fpgrowth(df.applymap(lambda x: 1 if x else 0), min_support=0.1, use_colnames=True)

In [13]:
frequent_itemsets

,support,itemsets
0,1.000000,(BUSINESSNO_TRANS)
1,1.000000,(REASON1)
2,1.000000,(CONTRA_NAME)
3,1.000000,(TR_SP_05)
4,1.000000,(REASON2)
...,...,...
262138,0.993089,"(VALUEDATE, CONTRA_NAME, FK_CURRENCY, TR_SP_05, TR_SP_01, TR_SP_03, BUSINESSNO_TRANS, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, ACC_CURRENCYISO, REASON2, ANALYTICAL_TRANS_CODE)"
262139,0.993089,"(VALUEDATE, CONTRA_NAME, FK_CURRENCY, REASON1, TR_SP_01, TR_SP_03, BUSINESSNO_TRANS, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, ACC_CURRENCYISO, REASON2, ANALYTICAL_TRANS_CODE)"
262140,0.993089,"(VALUEDATE, CONTRA_NAME, REASON1, TR_SP_05, TR_SP_01, TR_SP_03, BUSINESSNO_TRANS, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, ACC_CURRENCYISO, REASON2, ANALYTICAL_TRANS_CODE)"
262141,0.993089,"(VALUEDATE, CONTRA_NAME, FK_CURRENCY, REASON1, TR_SP_05, TR_SP_01, TR_SP_03, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, ACC_CURRENCYISO, REASON2, ANALYTICAL_TRANS_CODE)"


In [14]:
# Generate association rules with a confidence threshold of 1.0 (100%)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=1.0)

In [15]:
rules = rules.sort_values(by=['confidence', 'lift'], ascending=False)
rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1181398,"(AMOUNT, CONTRA_NAME, FK_CURRENCY, TR_SP_04)","(TR_SP_05, ACC_CURRENCYISO, TR_SP_01, TR_SP_02, AMOUNTORIG, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
1181399,"(AMOUNT, CONTRA_NAME, ACC_CURRENCYISO, TR_SP_04)","(TR_SP_05, FK_CURRENCY, TR_SP_01, TR_SP_02, AMOUNTORIG, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
1181400,"(AMOUNT, TR_SP_05, FK_CURRENCY, TR_SP_04)","(CONTRA_NAME, ACC_CURRENCYISO, TR_SP_01, TR_SP_02, AMOUNTORIG, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
1181401,"(AMOUNT, TR_SP_05, ACC_CURRENCYISO, TR_SP_04)","(CONTRA_NAME, FK_CURRENCY, TR_SP_01, TR_SP_02, AMOUNTORIG, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
1181402,"(AMOUNTORIG, CONTRA_NAME, FK_CURRENCY, TR_SP_04)","(AMOUNT, TR_SP_05, ACC_CURRENCYISO, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
...,...,...,...,...,...,...,...,...,...,...
3705279,"(VALUEDATE, CONTRA_NAME, ACC_CURRENCYISO, TR_SP_01, TR_SP_03, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, REASON1, REASON2, ANALYTICAL_TRANS_CODE)",(BUSINESSNO_TRANS),0.993089,1.0000,0.993089,1.0,1.000000,8.755912e-08,inf,0.000013
3705293,"(VALUEDATE, TR_SP_05, CONTRA_NAME, FK_CURRENCY, TR_SP_01, TR_SP_03, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, ACC_CURRENCYISO, REASON2, ANALYTICAL_TRANS_CODE)",(BUSINESSNO_TRANS),0.993089,1.0000,0.993089,1.0,1.000000,8.755912e-08,inf,0.000013
3705310,"(VALUEDATE, CONTRA_NAME, FK_CURRENCY, ACC_CURRENCYISO, TR_SP_01, TR_SP_03, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, REASON1, REASON2, ANALYTICAL_TRANS_CODE)",(BUSINESSNO_TRANS),0.993089,1.0000,0.993089,1.0,1.000000,8.755912e-08,inf,0.000013
3705314,"(VALUEDATE, TR_SP_05, CONTRA_NAME, ACC_CURRENCYISO, TR_SP_01, TR_SP_03, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, REASON1, REASON2, ANALYTICAL_TRANS_CODE)",(BUSINESSNO_TRANS),0.993089,1.0000,0.993089,1.0,1.000000,8.755912e-08,inf,0.000013


**Customer and Transaction Attributes**

In [62]:
# Define the itemsets of interest
itemsets_of_interest = {'CUST_CUSTNO', 'BUSINESSNO_TRANS', 'VALUEDATE', 'AMOUNT', 'ACC_BUSINESSTYPE', 'TR_FLAG_01'}

In [63]:
# Function to check if itemsets contain any of the specified itemsets
def contains_itemsets(antecedents, consequents, itemsets):
    return any(item in itemsets for item in antecedents) and any(item in itemsets for item in consequents)

In [65]:
# Filter rules
filtered_rules = rules[rules.apply(lambda row: contains_itemsets(row['antecedents'], row['consequents'], itemsets_of_interest), axis=1)]


In [66]:
filtered_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1185532,"(CONTRA_NAME, FK_CURRENCY, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG)","(AMOUNT, TR_SP_05, ACC_CURRENCYISO, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
1185539,"(CONTRA_NAME, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG, ACC_CURRENCYISO)","(AMOUNT, TR_SP_05, FK_CURRENCY, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
1185544,"(TR_SP_05, FK_CURRENCY, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG)","(AMOUNT, CONTRA_NAME, ACC_CURRENCYISO, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
1185550,"(TR_SP_05, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG, ACC_CURRENCYISO)","(AMOUNT, CONTRA_NAME, FK_CURRENCY, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
1185553,"(AMOUNT, CONTRA_NAME, FK_CURRENCY, TR_SP_04)","(TR_SP_05, ACC_CURRENCYISO, TR_SP_01, BUSINESSNO_TRANS, TR_SP_02, AMOUNTORIG, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,1.249283e-03,inf,0.011134
...,...,...,...,...,...,...,...,...,...,...
3705279,"(VALUEDATE, CONTRA_NAME, ACC_CURRENCYISO, TR_SP_01, TR_SP_03, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, REASON1, REASON2, ANALYTICAL_TRANS_CODE)",(BUSINESSNO_TRANS),0.993089,1.0000,0.993089,1.0,1.000000,8.755912e-08,inf,0.000013
3705293,"(VALUEDATE, TR_SP_05, CONTRA_NAME, FK_CURRENCY, TR_SP_01, TR_SP_03, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, ACC_CURRENCYISO, REASON2, ANALYTICAL_TRANS_CODE)",(BUSINESSNO_TRANS),0.993089,1.0000,0.993089,1.0,1.000000,8.755912e-08,inf,0.000013
3705310,"(VALUEDATE, CONTRA_NAME, FK_CURRENCY, ACC_CURRENCYISO, TR_SP_01, TR_SP_03, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, REASON1, REASON2, ANALYTICAL_TRANS_CODE)",(BUSINESSNO_TRANS),0.993089,1.0000,0.993089,1.0,1.000000,8.755912e-08,inf,0.000013
3705314,"(VALUEDATE, TR_SP_05, CONTRA_NAME, ACC_CURRENCYISO, TR_SP_01, TR_SP_03, TR_SP_02, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, REASON1, REASON2, ANALYTICAL_TRANS_CODE)",(BUSINESSNO_TRANS),0.993089,1.0000,0.993089,1.0,1.000000,8.755912e-08,inf,0.000013


In [68]:

# Sort the filtered rules by lift and confidence in descending order
top_10_rules = filtered_rules.sort_values(by=['lift', 'confidence'], ascending=False).head(10)

# Display the top 10 rules
print("Top 10 rules by lift and confidence:")
top_10_rules


Top 10 rules by lift and confidence:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1185532,"(CONTRA_NAME, FK_CURRENCY, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG)","(AMOUNT, TR_SP_05, ACC_CURRENCYISO, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134
1185539,"(CONTRA_NAME, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG, ACC_CURRENCYISO)","(AMOUNT, TR_SP_05, FK_CURRENCY, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134
1185544,"(TR_SP_05, FK_CURRENCY, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG)","(AMOUNT, CONTRA_NAME, ACC_CURRENCYISO, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134
1185550,"(TR_SP_05, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG, ACC_CURRENCYISO)","(AMOUNT, CONTRA_NAME, FK_CURRENCY, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134
1185553,"(AMOUNT, CONTRA_NAME, FK_CURRENCY, TR_SP_04)","(TR_SP_05, ACC_CURRENCYISO, TR_SP_01, BUSINESSNO_TRANS, TR_SP_02, AMOUNTORIG, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134
1185554,"(AMOUNT, CONTRA_NAME, ACC_CURRENCYISO, TR_SP_04)","(TR_SP_05, FK_CURRENCY, TR_SP_01, BUSINESSNO_TRANS, TR_SP_02, AMOUNTORIG, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134
1185555,"(AMOUNT, TR_SP_05, FK_CURRENCY, TR_SP_04)","(CONTRA_NAME, ACC_CURRENCYISO, TR_SP_01, BUSINESSNO_TRANS, TR_SP_02, AMOUNTORIG, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134
1185556,"(AMOUNT, TR_SP_05, ACC_CURRENCYISO, TR_SP_04)","(CONTRA_NAME, FK_CURRENCY, TR_SP_01, BUSINESSNO_TRANS, TR_SP_02, AMOUNTORIG, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134
1189411,"(CONTRA_NAME, FK_CURRENCY, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG, REASON2)","(AMOUNT, TR_SP_05, ACC_CURRENCYISO, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134
1189418,"(CONTRA_NAME, TR_SP_04, BUSINESSNO_TRANS, AMOUNTORIG, ACC_CURRENCYISO, REASON2)","(AMOUNT, TR_SP_05, FK_CURRENCY, TR_SP_01, TR_SP_02, REASON1, ANALYTICAL_TRANS_CODE)",0.128792,0.9903,0.128792,1.0,1.009795,0.001249,inf,0.011134


In [ ]:


# Generate association rules with a confidence threshold of 0.8
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)

# Sort the rules by confidence in descending order
rules = rules.sort_values(by=['confidence'], ascending=False)

# Display the rules
print("Association rules with confidence threshold of 0.8:")
rules


In [ ]:
# prompt: iloc data for antecedents_items = ['CONTRA_NAME', 'FK_CURRENCY', 'TR_SP_04', 'BUSINESSNO_TRANS', 'AMOUNTORIG']
# consequents_items = ['AMOUNT', 'TR_SP_05', 'ACC_CURRENCYISO', 'TR_SP_01', 'TR_SP_02', 'REASON1', 'ANALYTICAL_TRANS_CODE']

antecedents_items = ['CONTRA_NAME', 'FK_CURRENCY', 'TR_SP_04', 'BUSINESSNO_TRANS', 'AMOUNTORIG']
consequents_items = ['AMOUNT', 'TR_SP_05', 'ACC_CURRENCYISO', 'TR_SP_01', 'TR_SP_02', 'REASON1','REASON2' ,'ANALYTICAL_TRANS_CODE']
data_for_rules = df.iloc[:, df.columns.isin(antecedents_items + consequents_items)]
data_for_rules